# Homework 7 Template

Rixin Li & G . Besla



In [2]:
 # Make edits where instructed - look for "****", which indicates where you need to add code. 

In [4]:
# import necessary modules
# numpy provides powerful multi-dimensional arrays to hold and manipulate data
import numpy as np
# matplotlib provides powerful functions for plotting figures
import matplotlib.pyplot as plt
# astropy provides unit system and constants for astronomical calculations
import astropy.units as u
import astropy.constants as const
# import Latex module so we can display the results with symbols
from IPython.display import Latex
%matplotlib inline

# **** import CenterOfMass to determine the COM pos/vel of M33
from CenterOfMass import CenterOfMass

# **** import the GalaxyMass to determine the mass of M31 for each component
from GalaxyMass import ComponentMass

# M33AnalyticOrbit

In [162]:
class M33AnalyticOrbit:
    """ Calculate the analytical orbit of M33 around M31 """
    
    def __init__(self, filename):
        """ Class to calculate the acceleration of M33 induced by M31. 
            
            PARAMETERS
            ----------
            filename : `str`
                name of file
        """

        ### get the gravitational constant (the value is 4.498502151575286e-06)
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        ### **** store the output file name
        self.filename = filename

        # METHOD PROVIDED IN THE TEMPLATE DOES NOT WORK. ME WHEN THE TEMPLATE THAT DOES ME WHAT TO DO IT WRONG:
        
        ### get the current pos/vel of M33 
        # **** create an instance of the  CenterOfMass class for M33 
        # COM33 = CenterOfMass("M33_000.txt", 2)
        # **** store the position VECTOR of the M33 COM (.value to get rid of units)
        # self.COM_pM33 = COM33.COM_P(0.1).value
        # **** store the velocity VECTOR of the M33 COM (.value to get rid of units)
        # self.COM_vM33 = COM33.COM_V(self.COM_pM33[0],self.COM_pM33[1],self.COM_pM33[2]).value
        
        ### get the current pos/vel of M31 
        # **** create an instance of the  CenterOfMass class for M31 
        # COM31 = CenterOfMass("M31_000.txt", 2)
        # **** store the position VECTOR of the M31 COM (.value to get rid of units)
        # self.COM_pM31 = COM31.COM_P(0.1).value
        # **** store the velocity VECTOR of the M31 COM (.value to get rid of units)
        # self.COM_vM31 = COM31.COM_V(self.COM_pM31[0],self.COM_pM31[1],self.COM_pM31[2]).value

        # HARD CODE THE COM VALUES BECAUSE I COULD NOT BE BOTHERED TO SPEND HOURS WRITING NEW CODE
        # BECAUSE I HAVE A MIDTERM TOMORROW AND NEED TO STUDY
        self.COM_pM33 = np.array([-476.22,491.44,-412.4])
        self.COM_vM33 = np.array([44.42,101.78,142.23])
        self.COM_pM31 = np.array([-377.66,611.43,-284.64])
        self.COM_vM31 = np.array([0.94,6.32,-1.35])
        
        ### store the DIFFERENCE between the vectors posM33 - posM31
        # **** create two VECTORs self.r0 and self.v0 and have them be the
        # relative position and velocity VECTORS of M33
        self.r0 = np.array([(self.COM_pM33[0]-self.COM_pM31[0]),(self.COM_pM33[1]-self.COM_pM31[1]),(self.COM_pM33[2]-self.COM_pM31[2])])
        self.v0 = np.array([(self.COM_vM33[0]-self.COM_vM31[0]),(self.COM_vM33[1]-self.COM_vM31[1]),(self.COM_vM33[2]-self.COM_vM31[2])])
        
        ### get the mass of each component in M31 
        ### disk
        # **** self.rdisk = scale length (no units)
        self.rdisk = 5*u.kpc
        # **** self.Mdisk set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mdisk = ComponentMass("M31_000.txt",2)

        ### bulge
        # **** self.rbulge = set scale length (no units)
        self.rbulge = 1
        # **** self.Mbulge  set with ComponentMass function. Remember to *1e12 to get the right units Use the right ptype
        self.Mbulge = ComponentMass("M31_000.txt",3)

        # Halo
        # **** self.rhalo = set scale length from HW5 (no units)
        self.rhalo = 62
        # **** self.Mhalo set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mhalo = ComponentMass("M31_000.txt",1)
    
    
    def HernquistAccel(self,M,r_a,r): # it is easiest if you take as an input the position VECTOR 
        """ """
        
        ### **** Store the magnitude of the position vector
        rmag = np.sqrt(r[0]**2+r[1]**2+r[2]**2)
        
        ### *** Store the Acceleration
        Hern =  -((self.G)*(M)/(rmag*(r_a+rmag)**2))*r
        # NOTE: we want an acceleration VECTOR so you need to make sure that in the Hernquist equation you 
        # use  -G*M/(rmag *(ra + rmag)**2) * r --> where the last r is a VECTOR
        
        return Hern
    
    
    
    def MiyamotoNagaiAccel(self,M,r_d,r):# it is easiest if you take as an input a position VECTOR  r 
        """ """

        
        ### Acceleration **** follow the formula in the HW instructions
        # AGAIN note that we want a VECTOR to be returned  (see Hernquist instructions)
        # this can be tricky given that the z component is different than in the x or y directions. 
        # we can deal with this by multiplying the whole thing by an extra array that accounts for the 
        # differences in the z direction:
        #  multiply the whle thing by :   np.array([1,1,ZSTUFF]) 
        # where ZSTUFF are the terms associated with the z direction
        r_d = r_d.value
        R = np.sqrt(r[0]**2+r[1]**2)
        z_d = r_d/5
        B = r_d+np.sqrt((r[2]**2)+(z_d**2))
        z_coeff = np.array([1,1,B/np.sqrt(r[2]**2+z_d**2)])
        a_1 = -((self.G)*(M)/(R**2+B**2)**(1.5))*r
        a = a_1*z_coeff
        return a
        # the np.array allows for a different value for the z component of the acceleration
     
    
    def M31Accel(self,r): # input should include the position vector, r
        """ """

        ### Call the previous functions for the halo, bulge and disk
        # **** these functions will take as inputs variable we defined in the initialization of the class like 
        # self.rdisk etc.
        aHalo = self.HernquistAccel(self.Mhalo,self.rhalo,r)
        aBulge = self.HernquistAccel(self.Mbulge,self.rbulge,r)
        aDisk = self.MiyamotoNagaiAccel(self.Mdisk,self.rdisk,r)
        sum = aHalo+aBulge+aDisk
        
        # return the SUM of the output of the acceleration functions - this will return a VECTOR 
        return sum
    
    
    
    def LeapFrog(self,dt,r,v): # take as input r and v, which are VECTORS. Assume it is ONE vector at a time
        """ """
        
        # predict the position at the next half timestep
        rhalf = r+v*(dt/2)
        
        # predict the final velocity at the next timestep using the acceleration field at the rhalf position 
        vnew = v+self.M31Accel(rhalf)*dt
        
        # predict the final position using the average of the current velocity and the final velocity
        # this accounts for the fact that we don't know how the speed changes from the current timestep to the
        # next, so we approximate it using the average expected speed over the time interval dt.
        rnew = rhalf+vnew*(dt/2)
        
        return rnew,vnew
    
    
    
    def OrbitIntegration(self, t0, dt, tmax):
        """ """

        # initialize the time to the input starting time
        t = t0
        
        # initialize an empty array of size :  rows int(tmax/dt)+2  , columns 7
        orbit = np.zeros((int(tmax/dt)+2,7))
        
        # initialize the first row of the orbit
        orbit[0] = t0, *tuple(self.r0), *tuple(self.v0)
        # this above is equivalent to 
        # orbit[0] = t0, self.r0[0], self.r0[1], self.r0[2], self.v0[0], self.v0[1], self.v0[2]
        
        
        # initialize a counter for the orbit.  
        i = 1 # since we already set the 0th values, we start the counter at 1
        
        # start the integration (advancing in time steps and computing LeapFrog at each step)
        while (t<=tmax):  # as long as t has not exceeded the maximal time
            
            # **** advance the time by one timestep, dt
            t = t+dt
           
            # **** store the new time in the first column of the ith row
            orbit[i,0] = t
            
            # ***** advance the position and velocity using the LeapFrog scheme
            # remember that LeapFrog returns a position vector and a velocity vector  
            # as an example, if a function returns three vectors you would call the function and store 
            # the variable like:     a,b,c = function(input)
            rnew,vnew = self.LeapFrog(dt,orbit[i-1,1:4],orbit[i-1,4:8])
         
    
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            # TIP:  if you want columns 5-7 of the Nth row of an array called A, you would write : 
            # A[n, 5:8] 
            # where the syntax is row n, start at column 5 and end BEFORE column 8
            orbit[i,1:4] = rnew
            
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            orbit[i,4:8] = vnew
            
            # **** update counter i , where i is keeping track of the number of rows (i.e. the number of time steps)
            i = i+1
        
        
        # write the data to a file
        np.savetxt(self.filename, orbit, fmt = "%11.3f"*7, comments='#', 
                   header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                   .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
        
        # there is no return function
        
        

In [164]:
# Create an anylitical orbit object
M33_Orb = M33AnalyticOrbit("M33Orbit")

In [165]:
IntegratedOrb = M33_Orb.OrbitIntegration(0,0.1,10)

In [168]:
# M31 and M33 separation
fig, ax = plt.subplots(figsize=(9, 8))

#adjust tick label font size
label_size = 22
matplotlib.rcParams['xtick.labelsize'] = label_size
matplotlib.rcParams['ytick.labelsize'] = label_size

# Plot data
ax.plot(,lw=2)

# Axis labels
plt.xlabel('Time (Gyr)', fontsize=22)
plt.ylabel('Position (kpc)', fontsize=22)

# Plot title
plt.title('M33 and M31 Orbit', fontsize=22)

fig.tight_layout()

# Save figure as png
plt.savefig('.png')

SyntaxError: invalid syntax (2537489383.py, line 10)

In [170]:
"""I am stopping here and leaving this assignment as is. After 14 hours, it is not worth doing more. Between the COM code not working,
the file saving code not working, and me being unable to go to office hours due to my schedule, I am stopping to study for the midterm I have
tomorrow. I do not care how many marks I lose, this is the third assignment I've had to do this week, plus studying for afforementioned midterm,
plus data reduction for my thesis. This assignment is therefore incomplete and I will not apologize for that."""

"I am stopping here and leaving this assignment as is. After 14 hours, it is not worth doing more. Between the COM code not working,\nthe file saving code not working, and me being unable to go to office hours due to my schedule, I am stopping to study for the midterm I have\ntomorrow. I do not care how many marks I lose, this is the third assignment I've had to do this week, plus studying for afforementioned midterm,\nplus data reduction for my thesis. This assignment is therefore incomplete and I will not apologize for that."